# Authenticate Google Colab
Allows access to Google Cloud and Google Drive using your credentials

In [12]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


# Retrieve BigQuery as a DataFrame
[Getting started with BigQuery](https://colab.research.google.com/notebooks/bigquery.ipynb)

### via magics

In [0]:
%%bigquery --project cptsrewards-hrd df
SELECT 
  COUNT(*) as total_rows
FROM `bigquery-public-data.samples.gsod`

In [4]:
df.head()

,min_date,max_date
0,2019-10-01 00:00:03+00:00,2019-12-31 23:53:48+00:00


### via google-cloud-bigquery

In [0]:
from google.cloud import bigquery

project_id = 'cptsrewards-hrd'
client = bigquery.Client(project=project_id)

query_string = """
SELECT 
  COUNT(*) as total_rows
FROM `bigquery-public-data.samples.gsod`
"""

df = client.query(query_string).to_dataframe()

In [14]:
df.head(2)

,visit_id,overlapping,visit_dtime,dwelltime,device_uuid,category,sub_category,chain,slat,slon,distance,addr,city,state,zip,country,country_code,sic_code,sic_code_id,location_id,min_slat,max_slat,min_slon,max_slon,vlat,vlon,min_vlat,max_vlat,min_vlon,max_vlon,isa_mall,stores_at_loc,source,in_store,isa_visit
0,20191125_288198384709436f967c8636c3f56a8b_1574...,0,2019-11-25 15:17:00+00:00,75,28819838-4709-436F-967C-8636C3F56A8B,,,Aubuchon Hardware,44.259133,-72.576042,84.7,40 Main Street,Montpelier,VT,05602,,,,,,44.259034,44.259232,-72.576165,-72.575918,44.259890,-72.575919,44.259308,44.260437,-72.576920,-72.575653,0,1,stihl,0,0
1,20191010_767e2e687af81cab7c656bb19da00555_1570...,0,2019-10-10 16:10:57+00:00,166,767E2E68-7AF8-1CAB-7C65-6BB19DA00555,,,Aubuchon Hardware,43.837702,-70.440305,65.1,777 Roosevelt Trail,Windham,ME,04062,,,,,,43.837603,43.837801,-70.440429,-70.440181,43.838245,-70.440001,43.837952,43.838509,-70.440453,-70.439705,0,1,stihl,0,0
2,20191220_94dfdc2721cb43a1bb5ec4e654f22333_1576...,0,2019-12-20 17:23:15+00:00,60,94DFDC27-21CB-43A1-BB5E-C4E654F22333,,,Woodward Enterprises Inc.,39.448910,-75.720770,101.4,226 W Main St,Middletown,DE,19709,,,,,,39.448811,39.449009,-75.720894,-75.720646,39.449537,-75.719913,39.449459,39.449604,-75.719986,-75.719788,0,1,stihl,0,0
3,20191216_fc0aa98fa5414153aff4c91942ba44a3_1576...,0,2019-12-16 11:27:41+00:00,161,FC0AA98F-A541-4153-AFF4-C91942BA44A3,,,Gorham Ace Hardware,43.680250,-70.441160,138.0,57 Main Street,Gorham,ME,04038,,,,,,43.680151,43.680349,-70.441284,-70.441036,43.681482,-70.441371,43.680939,43.681808,-70.441681,-70.441139,0,1,stihl,0,0
4,20191012_4c45988e9fd3612cffe55c310edbc6db_1570...,0,2019-10-12 20:57:22+00:00,44,4C45988E-9FD3-612C-FFE5-5C310EDBC6DB,,,Annie's Ace Hardware - Brookland,38.931630,-76.994670,75.1,3405 8th Street Ne,Washington,DC,20017,,,,,,38.931531,38.931729,-76.994794,-76.994546,38.932261,-76.994980,38.931976,38.932358,-76.995644,-76.994713,0,1,stihl,0,0


### via pandas-gbq

In [0]:
import pandas as pd

project_id = 'cptsrewards-hrd'
query_string = """
SELECT 
  COUNT(*) as total_rows
FROM `bigquery-public-data.samples.gsod`
"""

df = pd.io.gbq.read_gbq(query_string, project_id=project_id)

In [26]:
df.head(2)

,visit_id,overlapping,visit_dtime,dwelltime,device_uuid,category,sub_category,chain,slat,slon,distance,addr,city,state,zip,country,country_code,sic_code,sic_code_id,location_id,min_slat,max_slat,min_slon,max_slon,vlat,vlon,min_vlat,max_vlat,min_vlon,max_vlon,isa_mall,stores_at_loc,source,in_store,isa_visit
0,20191125_288198384709436f967c8636c3f56a8b_1574...,0,2019-11-25 15:17:00+00:00,75,28819838-4709-436F-967C-8636C3F56A8B,,,Aubuchon Hardware,44.259133,-72.576042,84.7,40 Main Street,Montpelier,VT,05602,,,,,,44.259034,44.259232,-72.576165,-72.575918,44.259890,-72.575919,44.259308,44.260437,-72.576920,-72.575653,0,1,stihl,0,0
1,20191010_767e2e687af81cab7c656bb19da00555_1570...,0,2019-10-10 16:10:57+00:00,166,767E2E68-7AF8-1CAB-7C65-6BB19DA00555,,,Aubuchon Hardware,43.837702,-70.440305,65.1,777 Roosevelt Trail,Windham,ME,04062,,,,,,43.837603,43.837801,-70.440429,-70.440181,43.838245,-70.440001,43.837952,43.838509,-70.440453,-70.439705,0,1,stihl,0,0


# Google Sheets
[External data: Local files, Drive, Sheets, and Cloud Storage](https://colab.research.google.com/notebooks/io.ipynb)
<br>
[From Pandas to Google Sheets](https://www.danielecook.com/from-pandas-to-google-sheets/)

### Export/Update Google Sheets

In [0]:
# Authenticate gspread
import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [0]:
# Definte functions to transform pandas to google sheets
def iter_pd(df):
    for val in df.columns:
        yield val
    for row in df.to_numpy():
        for val in row:
            if pd.isna(val):
                yield ""
            else:
                yield val

def pandas_to_sheets(pandas_df, sheet, clear = True):
    # Casts all data as string to avoid loading to sheet problems
    df_string = pandas_df.astype('str')
    # Updates all values in a workbook to match a pandas dataframe
    if clear:
        sheet.clear()
    (row, col) = pandas_df.shape
    cells = sheet.range("A1:{}".format(gspread.utils.rowcol_to_a1(row + 1, col)))
    for cell, val in zip(cells, iter_pd(df_string)):
        cell.value = val
    sheet.update_cells(cells)

The spreadsheet key is the red part of the url as show below
<br>
![](https://miro.medium.com/max/1775/1*Hu8a8Lg2Zxyy_6LKjod_1Q.png)

In [0]:
# Set Workbook
wb = gc.open_by_key("example")
# Set Worksheet
target_sheet = wb.worksheet("testing")

pandas_to_sheets(df, target_sheet)

### Gspread Examples
[https://awesomeopensource.com/project/burnash/gspread](https://awesomeopensource.com/project/burnash/gspread)

#### Opening a Spreadsheet

In [0]:
# You can open a spreadsheet by its title as it appears in Google Docs
sh = gc.open('My poor gym results') # <-- Look ma, no keys!

# If you want to be specific, use a key (which can be extracted from
# the spreadsheet's url)
sht1 = gc.open_by_key('0BmgG6nO_6dprdS1MN3d3MkdPa142WFRrdnRRUWl1UFE')

# Or, if you feel really lazy to extract that key, paste the entire url
sht2 = gc.open_by_url('https://docs.google.com/spreadsheet/ccc?key=0Bm...FE&hl')

#### Creating a Spreadsheet

In [0]:
sh = gc.create('A new spreadsheet')

# But that new spreadsheet will be visible only to your script's account.
# To be able to access newly created spreadsheet you *must* share it

#### Selecting a Worksheet

In [0]:
# Select worksheet by index. Worksheet indexes start from zero
worksheet = sh.get_worksheet(0)

# By title
worksheet = sh.worksheet("January")

# Most common case: Sheet1
worksheet = sh.sheet1

# Get a list of all worksheets
worksheet_list = sh.worksheets()

#### Creating a Worksheet

In [0]:
worksheet = sh.add_worksheet(title="A worksheet", rows="100", cols="20")